In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ed47  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [5]:
bucket_name = '316431873' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [6]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)


In [7]:
# # adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_anc2.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [8]:
doc_anchor_text = parquetFile.select("id", "anchor_text").rdd


In [9]:
from inverted_anc2 import *

In [10]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  frq = {}
  ans = []
  for word in tokens: 
    if word not in all_stopwords: 
       if word not in frq.keys():
             frq[word] = 1
       else:
             frq[word] +=1 
  for key, val in frq.items():
      curr = (key, (id,val))
      ans.append(curr)
  return ans

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  ans = sorted(unsorted_pl, key= lambda x : x[0])
  return ans




def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda y : (y[0], len(y[1])))


In [12]:
NUM_BUCKETS = 200
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def token2bucket_id_anc(token):
  return (int(_hash(token),16) % NUM_BUCKETS) + 125


def partition_postings_and_write_anc(postings):
  ans =postings.map(lambda tup:  (token2bucket_id_anc(tup[0]) , tup))
  return   ans.groupByKey().map(lambda tup:  InvertedIndex_anc.write_a_posting_list(tup, bucket_name))

def anc_tup(x):
  id = x[0]
  lst = []
  for row in x[1]:
    s = ''
    s+= row['text']
    lst.append((row['id'],s))
  return lst





In [14]:
doc_anchor_text_pairs = doc_anchor_text.map(lambda x : anc_tup(x))
doc_anchor_text_pairs = doc_anchor_text_pairs.flatMap(lambda x: list(set(x)))

In [15]:

anchor_word_counts_for = doc_anchor_text_pairs.flatMap(lambda x: word_count(x[1], x[0]))


In [16]:
postings = anchor_word_counts_for.groupByKey().mapValues(reduce_word_counts)


In [17]:
posting_locs_list = partition_postings_and_write_anc(postings).collect()


In [18]:
posting_locs_list


[248,
 249,
 125,
 250,
 126,
 251,
 127,
 128,
 252,
 129,
 253,
 130,
 254,
 131,
 255,
 256,
 132,
 133,
 257,
 258,
 134,
 135,
 259,
 136,
 260,
 261,
 137,
 138,
 262,
 139,
 263,
 140,
 264,
 141,
 265,
 266,
 142,
 267,
 143,
 144,
 268,
 145,
 269,
 270,
 146,
 147,
 271,
 272,
 148,
 149,
 273,
 274,
 150,
 275,
 151,
 152,
 276,
 153,
 277,
 154,
 278,
 279,
 155,
 280,
 156,
 157,
 281,
 158,
 282,
 283,
 159,
 160,
 284,
 161,
 285,
 162,
 286,
 163,
 287,
 164,
 288,
 289,
 165,
 290,
 166,
 167,
 291,
 292,
 168,
 293,
 169,
 170,
 294,
 171,
 295,
 296,
 172,
 297,
 173,
 174,
 298,
 299,
 175,
 176,
 300,
 301,
 177,
 178,
 302,
 179,
 303,
 180,
 304,
 181,
 305,
 182,
 306,
 307,
 183,
 184,
 308,
 185,
 309,
 186,
 310,
 187,
 311,
 312,
 188,
 189,
 313,
 190,
 314,
 191,
 315,
 192,
 316,
 193,
 317,
 318,
 194,
 195,
 319,
 320,
 196,
 321,
 197,
 322,
 198,
 199,
 323,
 324,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214

In [19]:
w2df_text = calculate_df(postings)
w2df_dict = w2df_text.collectAsMap()

In [20]:
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anc'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [21]:
# Create inverted index instance
inverted = InvertedIndex_anc()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_anc')

In [22]:
# upload to gs
index_src = "index_anc.pkl"
index_dst = f'gs://{bucket_name}/indexses/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_anc.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 95.1 MiB/ 95.1 MiB]                                                
Operation completed over 1 objects/95.1 MiB.                                     


In [32]:
from google.cloud import storage
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1  
BLOCK_SIZE = 1999998


def post_index(word, index):
    client = storage.Client()
    bucket = client.bucket("316431873")
    b = []
    posting_list = [] 
    df = index.df[word]
    path = index.path
    with closing(MultiFileReader()) as reader:
        locs = index.posting_locs[word]
        n_bytes = index.df[word] * TUPLE_SIZE
#         b = reader.read(locs, n_bytes)
        for f_name, offset in locs:
                post_before = ""
                blob = bucket.blob(f"{path}/{f_name}")
                f = blob.open('rb')
#                 print(type(f))
                f.seek(offset)
                lst = [n_bytes , BLOCK_SIZE - offset]
                if lst[0] <= (BLOCK_SIZE - offset):
                    n_read = n_bytes
                else: 
                    n_read = BLOCK_SIZE - offset

                b.append(f.read(n_read))
                n_bytes -= n_read
                post_before = b''.join(b)
                for i in range(df):
                    doc_id = int.from_bytes(post_before[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                    tf = int.from_bytes(post_before[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                    posting_list.append((doc_id, tf))
        return list(set(posting_list))




In [26]:
inverted.path  = 'postings_gcp_anc'

In [33]:
post_index('air',inverted)

[(28566445, 1),
 (528652, 1),
 (1244901, 1),
 (408948, 1),
 (3648029, 1),
 (10585, 1),
 (34071, 1),
 (66101, 1),
 (57161, 1),
 (176737, 1),
 (342493, 1),
 (7791681, 1),
 (36170450, 1),
 (2379684, 1),
 (19450421, 1),
 (1265468, 1),
 (41327146, 1),
 (5665341, 1),
 (24629811, 1),
 (252948, 1),
 (15350030, 1),
 (132452, 1),
 (881523, 1),
 (47333817, 1),
 (33654163, 1),
 (20299080, 1),
 (63807881, 1),
 (22868884, 1),
 (3538081, 1),
 (4716357, 1),
 (61339773, 1),
 (10326536, 1),
 (9896539, 1),
 (43774059, 1),
 (616362, 1),
 (4992749, 1),
 (7515581, 1),
 (46439953, 1),
 (36675055, 1),
 (15934844, 1),
 (4705418, 1),
 (1497971, 1),
 (2015514, 1),
 (6599943, 1),
 (22228846, 1),
 (38647790, 1),
 (42968661, 1),
 (947874, 1),
 (4672468, 1),
 (41272318, 1),
 (22912541, 1),
 (637453, 1),
 (637057, 1),
 (2952639, 1),
 (3636462, 1),
 (18646790, 1),
 (33767090, 1),
 (37094113, 1),
 (48919292, 1),
 (57944168, 1),
 (22019201, 1),
 (715139, 1),
 (5895321, 1),
 (42989228, 1),
 (5982867, 1),
 (18611969, 1),
